### Intro
LIDAR and RADAR have their pros and cons, we combine both to get the best of them. Here we will try to build Extended Kalman Filters.  

### Lesson map


Build an **Extended** Kalman Filter: It is extended in the sense that it can handle **more complex motion and measurement models**

#### Process Flow
![img](./images/5-1.png)

#### Process Flow in Brief
 - We have 2 sensors - radar and lidar. These sensors are used to estimate state and velocity of a pedestrian. This state is represented by 2D position and 2D velocity.![img](./images/5-2.png)

Each time we receive a new measurement from the given sensor, the estimation function is triggered. At this point we perform 2 steps - State Prediction and Measurement Update.

In the Prediction step, we predict the pedestrian state and its covariance. We do so by taking into account the elapsed time between the current and previous observations. Let this time be Δt.

The measurement update depends on sensor type. There are 2 cases we will see - radar and lidar. If the current measurement is generated by laser, we apply standard Kalman filter. However, radar measurements involve a nonlinear measurement function, so we use different tweaks to handle measurement update, for eg. Extended Kalman filter equations.

#### Repeating the flow again


Imagine you are in a car equipped with sensors on the outside. The car sensors can detect objects moving around: for example, the sensors might detect a bicycle.

The Kalman Filter algorithm will go through the following steps:

- **first measurement** - the filter will receive initial measurements of the bicycle's position relative to the car. These measurements will come from a radar or lidar sensor.
- **initialize state and covariance matrices** - the filter will initialize the bicycle's position based on the first measurement.
- then the car will receive another sensor measurement after a time period Δt.
- **predict** - the algorithm will predict where the bicycle will be after time Δt. One basic way to predict the bicycle location after Δt is to assume the bicycle's velocity is constant; thus the bicycle will have moved velocity * Δt. In the extended Kalman filter lesson, we will assume the velocity is constant; in the unscented Kalman filter lesson, we will introduce a more complex motion model.
- **update** - the filter compares the "predicted" location with what the sensor measurement says. The predicted location and the measured location are combined to give an updated location. The Kalman filter will put more weight on either the predicted location or the measured location depending on the uncertainty of each value.
- then the car will receive another sensor measurement after a time period Δt. The algorithm then does another **predict** and **update** step.



### Lesson Variables and Equations
Refer the file - sensor-fusion-ekf-reference.pdf as cheat sheet for the lesson.


### Estimation Problem Refresh

This is how a single sensor flow works:

Predict step - Use information we have to predict the state of the pedestrian until the next measurement arrives
Measurement step - Use new observation to correct your belief of the state
![img](./images/5-3.png)

But what happens when we have 2 sensors that observe the same pedestrian, how does change the Kalman Filters?
Answer - Actually we can keep the same processing flow with the difference that each sensor is going to have its own prediction and update step, i.e. the belief about pedestrian's location and velocity is updated asynchronously.

#### Definition of Variables
x is Mean and covariance is P.

- x is the mean state vector. For an extended Kalman filter, the mean state vector contains information about the object's position and velocity that you are tracking. It is called the "mean" state vector because position and velocity are represented by a gaussian distribution with mean x.
- P is the state covariance matrix, which contains information about the uncertainty of the object's position and velocity. You can think of it as containing standard deviations.
- k represents time steps. So xk refers to the object's position and velocity vector at time k.
- The notation k+1∣k refers to the prediction step. At time k+1, you receive a sensor measurement. Before taking into account the sensor measurement to update your belief about the object's position and velocity, you predict where you think the object will be at time k+1. You can predict the position of the object at k+1 based on its position and velocity at time k. Hence xk+1∣k means that you have predicted where the object will be at k+1 but have not yet taken the sensor measurement into account.
- xk+1 means that you have now predicted where the object will be at time k+1 and then used the sensor measurement to update the object's position and velocity.

#### Example of flow
Now lets understand the flow:
![img](./images/5-4.png)

##### Received the LIDAR Update
- Now suppose that we are at time k+1 and we received Laser measurement received at time k+1, radar measurement received at time k+2.
- The first thing we do before we look at the measurement update is to make a prediction about where we think the pedestrian from time k will be at time k+1.
- If laser and radar measurements arrive at the same time, predict and update for one sensor first (e.g. laser) and then predict and update for the next sensor. The order doesn't matter.
- The second thing we do is the so called measurement update, where we combine the pedestrian's predicted state with the new laser measurement. What we now have is a more accurate belief about the pedestrian's position at time k+1,this is what we call the posterior.

##### Received the RADAR Update
 - Now let's imagine that we receive the radar measurement at time k+2. 
 - First, we again predict the pedestrian state from k+1 to k+2. Note, this prediction for radar is exactly the same function as in the laser case.
 - What changes, in this case, is the Measurement Update step. As we know, the radar sees the word differently than laser. 
 - While laser provides measurement in a Cartesian coordinate system, radar provides measurement in a polar coordinate system. Thus we have to use different measurement update functions specific to radar data, so this is a moredetailed view of the Kalman filter. 
 - We received the measurements from different sensors at each timestamp, and then we make a prediction followed by a measurement update.

 

### Kalman Filter Intuition
The Kalman equation contains many variables, so here is a high level overview to get some intuition about what the Kalman filter is doing.


## Kalman Filter in C++

In [ ]:
// Write a function 'filter()' that implements a multi-
// dimensional Kalman Filter for the example given
//============================================================================
#include <iostream>
#include "Eigen/Dense"
#include <vector>

using namespace std;
using namespace Eigen;

//Kalman Filter variables
VectorXd x;	// object state
MatrixXd P;	// object covariance matrix
VectorXd u;	// external motion
MatrixXd F; // state transition matrix
MatrixXd H;	// measurement matrix
MatrixXd R;	// measurement covariance matrix
MatrixXd I; // Identity matrix
MatrixXd Q;	// process covariance matrix

vector<VectorXd> measurements;
void filter(VectorXd &x, MatrixXd &P);


int main() {
	/**
	 * Code used as example to work with Eigen matrices
	 */
//	//you can create a  vertical vector of two elements with a command like this
//	VectorXd my_vector(2);
//	//you can use the so called comma initializer to set all the coefficients to some values
//	my_vector << 10, 20;
//
//
//	//and you can use the cout command to print out the vector
//	cout << my_vector << endl;
//
//
//	//the matrices can be created in the same way.
//	//For example, This is an initialization of a 2 by 2 matrix
//	//with the values 1, 2, 3, and 4
//	MatrixXd my_matrix(2,2);
//	my_matrix << 1, 2,
//			3, 4;
//	cout << my_matrix << endl;
//
//
//	//you can use the same comma initializer or you can set each matrix value explicitly
//	// For example that's how we can change the matrix elements in the second row
//	my_matrix(1,0) = 11;    //second row, first column
//	my_matrix(1,1) = 12;    //second row, second column
//	cout << my_matrix << endl;
//
//
//	//Also, you can compute the transpose of a matrix with the following command
//	MatrixXd my_matrix_t = my_matrix.transpose();
//	cout << my_matrix_t << endl;
//
//
//	//And here is how you can get the matrix inverse
//	MatrixXd my_matrix_i = my_matrix.inverse();
//	cout << my_matrix_i << endl;
//
//
//	//For multiplying the matrix m with the vector b you can write this in one line as let’s say matrix c equals m times v.
//	//
//	MatrixXd another_matrix;
//	another_matrix = my_matrix*my_vector;
//	cout << another_matrix << endl;


	//design the KF with 1D motion
	x = VectorXd(2);
	x << 0, 0;

	P = MatrixXd(2, 2);
	P << 1000, 0, 0, 1000;

	u = VectorXd(2);
	u << 0, 0;

	F = MatrixXd(2, 2);
	F << 1, 1, 0, 1;

	H = MatrixXd(1, 2);
	H << 1, 0;

	R = MatrixXd(1, 1);
	R << 1;

	I = MatrixXd::Identity(2, 2);

	Q = MatrixXd(2, 2);
	Q << 0, 0, 0, 0;

	//create a list of measurements
	VectorXd single_meas(1);
	single_meas << 1;
	measurements.push_back(single_meas);
	single_meas << 2;
	measurements.push_back(single_meas);
	single_meas << 3;
	measurements.push_back(single_meas);

	//call Kalman filter algorithm
	filter(x, P);

	return 0;

}


void filter(VectorXd &x, MatrixXd &P) {

	for (unsigned int n = 0; n < measurements.size(); ++n) {

		VectorXd z = measurements[n];
		//YOUR CODE HERE
		
		// KF Measurement update step
		 
		// new state
		
		// KF Prediction step
		
		std::cout << "x=" << std::endl <<  x << std::endl;
		std::cout << "P=" << std::endl <<  P << std::endl;


	}
}

In file included from input_line_6:4:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iostream:38:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/ios:215:
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iosfwd:176:14: error: reference to 'mbstate_t' is ambiguous
typedef fpos<mbstate_t>    streampos;
             ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/cwchar:119:9: note: candidate found by name lookup is 'std::__1::mbstate_t'
using ::mbstate_t;
        ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/cwchar:119:9: note: candidate found by name lookup is 'std::__1::mbstate_t'
using ::mbstate_t;
        ^
/Applications/Xcode.app/Contents/Developer/Toolc

/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:3771:16: error: invalid operands to binary expression ('const std::type_info' and 'const std::type_info')
    return __t == typeid(_Dp) ? _VSTD::addressof(__data_.first().second()) : 0;
           ~~~ ^  ~~~~~~~~~~~
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/typeinfo:128:10: note: candidate function not viable: no known conversion from 'const std::type_info' to 'const std::type_info' for 1st argument
    bool operator==(const type_info& __arg) const _NOEXCEPT
         ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/system_error:558:1: note: candidate function not viable: no known conversion from 'const std::type_info' to 'const std::__1::error_code' for 1st argument
operator==(const error_code& __x, const error_code& __y) _NOEXCEPT
^
/Applications/Xcode.app

/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:3029:1: note: candidate template ignored: could not match 'unique_ptr<type-parameter-0-0, type-parameter-0-1>' against 'const std::type_info'
operator==(nullptr_t, const unique_ptr<_T1, _D1>& __x) _NOEXCEPT
^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:4873:1: note: candidate template ignored: could not match 'shared_ptr<type-parameter-0-0>' against 'const std::type_info'
operator==(const shared_ptr<_Tp>& __x, const shared_ptr<_Up>& __y) _NOEXCEPT
^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:4922:1: note: candidate template ignored: could not match 'shared_ptr<type-parameter-0-0>' against 'const std::type_info'
operator==(const shared_ptr<_Tp>& __x, nullptr_t) _NOEXCEPT
^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xct

/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:4873:1: note: candidate template ignored: could not match 'shared_ptr<type-parameter-0-0>' against 'const std::type_info'
operator==(const shared_ptr<_Tp>& __x, const shared_ptr<_Up>& __y) _NOEXCEPT
^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:4922:1: note: candidate template ignored: could not match 'shared_ptr<type-parameter-0-0>' against 'const std::type_info'
operator==(const shared_ptr<_Tp>& __x, nullptr_t) _NOEXCEPT
^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:4930:1: note: candidate template ignored: could not match 'shared_ptr<type-parameter-0-0>' against 'const std::type_info'
operator==(nullptr_t, const shared_ptr<_Tp>& __x) _NOEXCEPT
^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../inclu

/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:4930:1: note: candidate template ignored: could not match 'shared_ptr<type-parameter-0-0>' against 'const std::type_info'
operator==(nullptr_t, const shared_ptr<_Tp>& __x) _NOEXCEPT
^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/string:493:6: note: candidate template ignored: could not match 'fpos<type-parameter-0-0>' against 'const std::type_info'
bool operator==(const fpos<_StateT>& __x, const fpos<_StateT>& __y)
     ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/string:3772:1: note: candidate template ignored: could not match 'basic_string<type-parameter-0-0, type-parameter-0-1, type-parameter-0-2>' against
      'const std::type_info'
operator==(const basic_string<_CharT, _Traits, _Allocator>& __lhs,
^
/Applications/Xcode.app/Contents/Developer/Toolchai

In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/__locale:15:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/string:439:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/algorithm:628:
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:5450:11: error: functions that differ only in their return type cannot be overloaded
__sp_mut& __get_sp_mut(const void*);
~~~~~~~~~ ^
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/memory:5446:39: note: previous declaration is here
    friend _LIBCPP_FUNC_VIS __sp_mut& __get_sp_mut(const void*);
                            ~~~~~~~~~ ^
In file included from input_line_6:4:
In file included from /Application

In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/iostream:38:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/ios:216:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/__locale:15:
In file included from /Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/string:439:
/Applications/Xcode.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/../include/c++/v1/algorithm:1466:5: error: unexpected namespace name '__1': expected expression
    _D1 __len1 = __last1 - __first1;
    ^
fatal error: too many errors emitted, stopping now [-ferror-limit=]
